In [3]:
%run setup_path.py
%load_ext autoreload
%autoreload 2

import pandas as pd

from os import path
from gorbslam.orbslam_processor import ModelType, ORBSLAMProcessor
from evo.core.metrics import StatisticsType

# TRAJECTORY_NAME = 'vabadussild_novatel'
# TRAJECTORY_NAME = 'vabadusepst_novatel'
TRAJECTORY_NAME = 'oldtown_novatel'

trajectories = [
    'vabadussild_novatel',
    'vabadusepst_novatel',
    'oldtown_novatel',
]

model_types = [
    ModelType.NN,
    ModelType.GBR,
    ModelType.RFR,
    ModelType.SVR,
    ModelType.UMEYAMA,
]

processors: dict[tuple[str, str], ORBSLAMProcessor] = {}
for trajectory_name in trajectories:
    for model_type in model_types:
        print(f"Processing {trajectory_name} with {model_type.name}")
        processors[(trajectory_name, model_type)] = ORBSLAMProcessor(path.join('~/orbslam-out', trajectory_name), model_type)
        processors[(trajectory_name, model_type)].initialize_model()
        processors[(trajectory_name, model_type)].fit_trajectories()
        processors[(trajectory_name, model_type)].save_trajectories()



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Processing vabadussild_novatel with NN
131/131 [==============================] - 0s 606us/step
Processing vabadussild_novatel with GBR
Processing vabadussild_novatel with RFR


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

Processing vabadussild_novatel with SVR
Processing vabadussild_novatel with UMEYAMA
Processing vabadusepst_novatel with NN
88/88 [==============================] - 0s 2ms/step
Processing vabadusepst_novatel with GBR
Processing vabadusepst_novatel with RFR
Processing vabadusepst_novatel with SVR


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


Processing vabadusepst_novatel with UMEYAMA
Processing oldtown_novatel with NN
28/28 [==============================] - 0s 902us/step
Processing oldtown_novatel with GBR
Processing oldtown_novatel with RFR


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

Processing oldtown_novatel with SVR
Processing oldtown_novatel with UMEYAMA


In [6]:
df = pd.DataFrame(
    columns=[
        "trajectory",
        "model",
        "loc",
        "min",
        "max",
        "mean",
        "median",
        "std",
        "rmse",
        "sse",
    ]
)
for (trajectory_name, model_type), processor in processors.items():
    print(f"Calculating APE for {trajectory_name} with {model_type.name}")
    ape = processor.calculate_ape_metric_all()
    for loc, ape_error in ape.items():
        df = df.append(
            {
                "trajectory": trajectory_name,
                "model": model_type.name,
                "loc": loc,
                "min": ape_error.get_statistic(StatisticsType.min),
                "max": ape_error.get_statistic(StatisticsType.max),
                "mean": ape_error.get_statistic(StatisticsType.mean),
                "median": ape_error.get_statistic(StatisticsType.median),
                "std": ape_error.get_statistic(StatisticsType.std),
                "rmse": ape_error.get_statistic(StatisticsType.rmse),
                "sse": ape_error.get_statistic(StatisticsType.sse),
            },
            ignore_index=True,
        )

df.to_csv("all_ape.csv", index=False)


Calculating APE for vabadussild_novatel with NN
Calculating APE for vabadussild_novatel with GBR


/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391708284.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391708284.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391708284.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391708284.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391

Calculating APE for vabadussild_novatel with RFR
Calculating APE for vabadussild_novatel with SVR


/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391708284.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391708284.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391708284.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391708284.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391

Calculating APE for vabadussild_novatel with UMEYAMA
Calculating APE for vabadusepst_novatel with NN


...

Calculating APE for oldtown_novatel with SVR


/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391708284.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391708284.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391708284.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391708284.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391

Calculating APE for oldtown_novatel with UMEYAMA


/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391708284.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391708284.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391708284.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391708284.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/x5/ssd1ylz90_39qqrwvg391k3w0000gp/T/ipykernel_37085/391

,trajectory,model,loc,min,max,mean,median,std,rmse,sse
0,vabadussild_novatel,NN,mapping,0.034549,4.929593,1.180200,0.987161,0.828306,1.441861,2.492676e+03
1,vabadussild_novatel,NN,0,0.022039,4.933677,1.100122,0.930663,0.749813,1.331348,1.623953e+04
2,vabadussild_novatel,NN,1,1.016700,275.178013,2.763229,2.155706,10.172941,10.541545,8.023165e+04
3,vabadussild_novatel,NN,2,0.045520,268.694466,1.334022,1.260450,4.192832,4.399938,8.067085e+04
4,vabadussild_novatel,GBR,mapping,1.450975,74.616503,17.921753,14.955385,10.525310,20.783921,5.179337e+05
5,vabadussild_novatel,GBR,0,0.749821,80.960593,16.860167,14.003519,11.003660,20.133201,3.713778e+06
6,vabadussild_novatel,GBR,1,0.497238,255.933660,6.497903,6.023473,9.730233,11.700435,9.884193e+04
7,vabadussild_novatel,GBR,2,3.134087,249.424001,14.395779,12.632795,11.306560,18.305102,1.396265e+06
8,vabadussild_novatel,RFR,mapping,0.007382,54.916843,1.837743,0.555917,4.411277,4.778772,2.738116e+04
9,vabadussild_novatel,RFR,0,0.007921,92.050899,2.260026,0.591528,5.983206,6.395817,3.747851e+05
